In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import calculate_bollinger_bands, calculate_rsi, calculate_daily_volatility
from utilities import print_title, print_label, print_footer
from utilities import generate_trading_signals
from utilities import fetch_and_download_sp500_data

In [2]:
# Import libraries for data analysis and visualization
import pandas as pd
import numpy as np
import yfinance as yf # Yahoo Finance data retrieval

# Import libraries for signal processing and peak detection
from scipy.signal import find_peaks

# Import libraries for plotting and visualization
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh') # Use Bokeh as the backend for Holoviews

# Import libraries for data storage and retrieval
from pathlib import Path

In [3]:
# Set display options for Pandas (optional, but often helpful)
pd.set_option('display.float_format', lambda x: '%.3f' % x) # Suppress scientific notation

In [4]:
data = fetch_and_download_sp500_data(start_date="2024-01-01")

[*********************100%***********************]  501 of 501 completed


╔═══════════════════════════════════════════════════════════════╗
║            Download Report for S&P 500 `adj close`            ║
╠═══════════════════════════════════════════════════════════════╣
║ Total Requested Tickers:       |             501              ║
║ Total Downloaded Tickers:      |             501              ║
║ Requested Date Range:          |   2024-01-01 to 2024-10-21   ║
║ Downloaded Date Range:         |   2024-01-02 to 2024-10-18   ║
║                       S&P 500 Data Downloaded Successfully... ║
╚═══════════════════════════════════════════════════════════════╝


In [5]:
data

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-02 00:00:00+00:00,138.750,185.640,154.184,134.480,109.850,75.550,346.920,580.070,191.157,72.760,...,239.950,33.986,93.825,63.470,102.360,113.670,127.125,121.390,267.980,196.570
2024-01-03 00:00:00+00:00,131.160,184.250,154.801,133.420,109.520,76.940,337.920,571.790,186.595,73.130,...,240.630,33.273,93.260,63.580,103.220,111.560,127.194,120.470,252.520,192.930
2024-01-04 00:00:00+00:00,131.000,181.910,155.766,133.720,110.980,77.470,337.090,567.050,183.741,71.750,...,239.490,32.814,93.438,63.730,102.320,112.330,126.790,120.200,252.970,194.040
2024-01-05 00:00:00+00:00,130.560,181.180,156.422,135.980,110.800,76.810,336.620,564.600,184.215,70.810,...,238.870,32.902,94.866,63.730,102.630,112.150,126.446,119.980,252.690,194.850
2024-01-08 00:00:00+00:00,133.380,185.560,155.737,140.080,112.400,77.120,340.350,580.550,186.615,71.060,...,242.170,33.263,96.255,63.580,100.920,112.850,126.741,121.970,261.090,196.150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-14 00:00:00+00:00,144.250,231.300,194.100,134.790,117.250,111.090,364.600,509.650,236.430,57.910,...,292.650,32.590,103.950,63.020,124.080,137.240,134.650,104.830,377.680,192.650
2024-10-15 00:00:00+00:00,144.580,233.850,191.860,133.270,116.050,108.060,368.660,508.030,225.870,57.670,...,291.140,33.050,99.220,63.430,120.350,136.680,133.220,104.600,375.300,192.390
2024-10-16 00:00:00+00:00,139.110,231.780,190.460,135.150,117.820,107.470,372.260,502.540,227.490,58.290,...,291.300,33.450,100.240,64.010,120.660,137.460,133.350,105.460,375.500,194.440
